# **Introdução ao Conceito de Árvore de Decisão**

Uma árvore de decisão é uma estrutura de modelagem não linear, que simula o processo de decisão com base em respostas a perguntas realizadas sequencialmente aos dados. Cada resposta é interpretada como um nó de divisão no algoritmo, e geometricamente representa uma divisão ortogonal a $n-1$ eixos das variáveis de saída.

Exemplo de uma estrutura de árvore de decisão:

<img src="https://i1.wp.com/www.vooo.pro/insights/wp-content/uploads/2016/12/RDS-Vooo_insights-Tutorial_arvore_de_decisao_02.jpg?resize=640%2C371&ssl=1" width=600>

A próxima imagem apresenta de forma ilustrativa o que acontece com o espaço de atributos.

<img src=https://paulvanderlaken.files.wordpress.com/2020/03/readme-titanic_plot-11.png width=500>

## **Processo de Aprendizado da Árvore de Decisão**

A primeira pergunta que uma árvore precisa responder é: *qual o atributo receberá a divisão?*

Dependendo do atributo que for utilizado no nó raiz, a árvore pode atingir um determinado grau de separação dos conjuntos de cada classe. Então deve existir uma métrica que direciona a separação para um determinado atributo.

<img src="https://i2.wp.com/www.vooo.pro/insights/wp-content/uploads/2016/12/RDS-Vooo-Tutorial_completo_arvore_decisao_03.jpg?resize=617%2C293&ssl=1" width=500>



### *Critério de Gini*

A impureza de Gini mede o quaõ "impuras" são as folhas de uma árvore contruídas após as divisões do nó. É calculada como:

$$Gini(D) = 1- \sum_{i=1}^{n}p_i$$

Essencialmente, a árvode de decisão compara a impureza de Gini antes e depois da realização das divisões em cada atributo, e seleciona aquele atributo que vai proporcionar a **maior purificação** da amostra.

### *Critério de Entropia*

A entropia é uma quantidade definida em física, engenharia e teoria da informação, com o objetivo de quantificar o **grau de desordem** de um sistema ou, de forma equivalente, o quanto se possui de informação a respeito de um sistema. É calculada como:

$$E = -\sum_{i=1}^{n}p_i \log_2{p_i}$$

De mesma forma, a árvore de decisão procura aquele atributo que proporciona a maior queda de entropia, que representa o maior ganho de informação.

In [43]:
import warnings
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

# ignorar warnings
warnings.filterwarnings('ignore')

In [3]:
# montando drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
# procurando arquivos no Drive
data = pd.read_csv('/content/drive/MyDrive/Bootcamp_DataScience/Algoritmos de Inteligência Artificial para Classificação/datasets/german_credit.csv')
data.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [5]:
# modificando os nomes das colunas
data.columns = data.columns.str.lower().str.replace(' ', '_')

In [6]:
# separando x e y - vamos usar todos as colunas
x = data.drop(['creditability'], axis=1)
y = data[['creditability']]

In [7]:
# separando treino e teste - com estratificação
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

In [9]:
# treinando nossa primeira árvore de decisão - não precisa escalonamento
dt = DecisionTreeClassifier().fit(x_train, y_train)

# realizando novas previsões
yhat_train = dt.predict(x_train)
yhat_test = dt.predict(x_test)

In [10]:
# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       210
           1       1.00      1.00      1.00       490

    accuracy                           1.00       700
   macro avg       1.00      1.00      1.00       700
weighted avg       1.00      1.00      1.00       700



In [11]:
# análise do desempenho
print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.50      0.61      0.55        90
           1       0.82      0.74      0.78       210

    accuracy                           0.70       300
   macro avg       0.66      0.67      0.66       300
weighted avg       0.72      0.70      0.71       300



Claramente houve um *overfitting* desse modelo. Ele foi capaz de modelar até os ruídos e variabilidades aleatórias do conjunto de dados, mas não foi capaz de reproduzir essas correlações no conjunto de teste.

Inevitavelmente, se fizermos perguntas o suficiente, vamos conseguir modelar qualquer sistema. No entanto, isso vai tornar o modelo tão complexo e específico, que dificilmente os padrões aprendidos serão reproduzidos em outros conjuntos de dados.

In [12]:
# Analisando as dimensões da árvore criada
print(f'Profundidade da árvore: {dt.get_depth()}')
print(f'Número de folhas da árvore: {dt.get_n_leaves()}')

Profundidade da árvore: 19
Número de folhas da árvore: 140


Temos um modelo extremamente profundo e complexo. Isso não somente dificulta a capacidade de generalização dos dados, mas também a explicação dos resultados do modelo para pessoas não técnicas.

Precisamos de uma forma de diminuir a profundidade da árvore, mas a questão é: qual a melhor profundidade?

Esse é um típico problema de **determinação (tuning) de hiperparâmetros**.

## **Determinação de Hiperparâmetros por Validação Cruzada**

Um dos desafios de se determinar os valores corretos de hiperparâmetros é que tais valores podem ser dependentes também dos dados que estamos submetendo o modelo. Não necessariamente um conjunto de hiperparâmetros que realizem uma boa previsão num conjunto de dados terá um bom desempenho em outro conjunto de dados.

Assim precisamos buscar um conjunto de hiperparâmetros que apresentem um bom desempenho *médio*, ou seja, que apresentem desempenhos adequados em diversas bases de dados distintas.

Para isso, podemos usar o conceito de **validação cruzada em $k$ folhas**. Veja um exemplo abaixo:

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*AAwIlHM8TpAVe4l2FihNUQ.png">

Na validação cruzada de $k$ folhas, o modelo é treinado $k$ vezes, sendo que em cada vez, uma fração de $1/k$ do conjunto de dados é utilizado como base de teste, enquanto as outras $(k-1)/k$ porções são utilizadas como base de treinamento.

Ao final do treinamento, escolhemos aquele conjunto de dados que resultam no melhor desempenho médio.

In [36]:
# criando um pipeline de modelagem
pipe = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=2))
])

# configurando um espaço de busca
params_grid = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': range(2, 11)
}

# configurando um amostrador de k folhas de forma estratificada
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)

# configurando um buscador de hiperparâmetros
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=params_grid,
    scoring='precision_weighted',
    cv=splitter,
    refit=True,
    error_score=0,
    verbose=10
)

In [37]:
# realizando busca
grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV 1/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 1/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=0.709 total time=   0.0s
[CV 2/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 2/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=0.709 total time=   0.0s
[CV 3/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 3/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=0.686 total time=   0.0s
[CV 4/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 4/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=0.658 total time=   0.0s
[CV 5/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 5/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=0.753 total time=   0.0s
[CV 6/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 6/

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=2, shuffle=True),
             error_score=0,
             estimator=Pipeline(steps=[('dt',
                                        DecisionTreeClassifier(random_state=2))]),
             param_grid={'dt__criterion': ['gini', 'entropy'],
                         'dt__max_depth': range(2, 11)},
             scoring='precision_weighted', verbose=10)

In [38]:
# analisando a melhor combinação de parâmetros
grid_search.best_params_

{'dt__criterion': 'gini', 'dt__max_depth': 2}

In [39]:
# analisando o melhor desempenho médio
grid_search.best_score_

0.7271531760016708

In [40]:
# analisando os desempenhos obtidos
precision_cv = []
for i in range(5):
  precision_cv.append(grid_search.cv_results_[f'split{i}_test_score'][np.where(grid_search.cv_results_['rank_test_score']==1)][0])

precision_cv

[0.7085714285714286,
 0.7089285714285715,
 0.6856702619414483,
 0.6583333333333332,
 0.7527333894028595]

In [41]:
# analisando o desempenho final
# realizando novas previsões
yhat_train = grid_search.best_estimator_.predict(x_train)
yhat_test = grid_search.best_estimator_.predict(x_test)

# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.51      0.80      0.62       210
           1       0.89      0.67      0.76       490

    accuracy                           0.71       700
   macro avg       0.70      0.73      0.69       700
weighted avg       0.77      0.71      0.72       700

Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.42      0.62      0.50        90
           1       0.80      0.64      0.71       210

    accuracy                           0.63       300
   macro avg       0.61      0.63      0.61       300
weighted avg       0.69      0.63      0.65       300



E se usarmos uma métrica mais voltada aos negócios para determinar o melhor conjunto de hiperparâmetros?

In [42]:
def customer_profit(yreal, ypred):

  # calcula a matriz de confusão
  conf_matrix = confusion_matrix(yreal, ypred)

  # extrai pontuações
  TP = conf_matrix[1, 1]  # Verdadeiros positivos
  FP = conf_matrix[0, 1]  # Falsos positivos
  TN = conf_matrix[0, 0]  # Verdadeiros negativos
  FN = conf_matrix[1, 0]  # Falsos negativos

  # calcula custo
  return ((50 * TP) + (-5 * FN) + (-5 * TN) + (-150 * FP)) / yreal.shape[0]

In [45]:
# configurando um buscador de hiperparâmetros
grid_search_profit = GridSearchCV(
    estimator=pipe,
    param_grid=params_grid,
    scoring=make_scorer(customer_profit, greater_is_better=True),
    cv=splitter,
    refit=True,
    error_score=-10,
    verbose=10
)

# realizando busca
grid_search_profit.fit(x_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV 1/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 1/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=5.357 total time=   0.0s
[CV 2/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 2/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=1.857 total time=   0.0s
[CV 3/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 3/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=-1.500 total time=   0.0s
[CV 4/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 4/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=-1.286 total time=   0.0s
[CV 5/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV 5/10; 1/18] END dt__criterion=gini, dt__max_depth=2;, score=10.071 total time=   0.0s
[CV 6/10; 1/18] START dt__criterion=gini, dt__max_depth=2.......................
[CV

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=2, shuffle=True),
             error_score=-10,
             estimator=Pipeline(steps=[('dt',
                                        DecisionTreeClassifier(random_state=2))]),
             param_grid={'dt__criterion': ['gini', 'entropy'],
                         'dt__max_depth': range(2, 11)},
             scoring=make_scorer(customer_profit), verbose=10)

In [47]:
# analisando a melhor combinação de parâmetros
grid_search_profit.best_params_

{'dt__criterion': 'gini', 'dt__max_depth': 2}

In [48]:
# analisando o melhor desempenho médio
grid_search_profit.best_score_

5.992857142857144

In [49]:
# analisando os desempenhos obtidos
profit_cv = []
for i in range(5):
  profit_cv.append(grid_search_profit.cv_results_[f'split{i}_test_score'][np.where(grid_search_profit.cv_results_['rank_test_score']==1)][0])

profit_cv

[5.357142857142857,
 1.8571428571428572,
 -1.5,
 -1.2857142857142858,
 10.071428571428571]

In [51]:
# analisando o desempenho final
# realizando novas previsões
yhat_train = grid_search_profit.best_estimator_.predict(x_train)
yhat_test = grid_search_profit.best_estimator_.predict(x_test)

# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))
print(customer_profit(y_train, yhat_train))

print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))
print(customer_profit(y_test, yhat_test))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.51      0.80      0.62       210
           1       0.89      0.67      0.76       490

    accuracy                           0.71       700
   macro avg       0.70      0.73      0.69       700
weighted avg       0.77      0.71      0.72       700

11.914285714285715
Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.42      0.62      0.50        90
           1       0.80      0.64      0.71       210

    accuracy                           0.63       300
   macro avg       0.61      0.63      0.61       300
weighted avg       0.69      0.63      0.65       300

3.1333333333333333
